In [1]:
%matplotlib inline
#from bokeh.io import output_notebook
#rom bokeh.plotting import *
from matmodlab2 import *
from numpy import *
#output_notebook()
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import dill
import sys
from numpy import cos, sin
from sklearn.linear_model import LinearRegression
from wccm_support import plot_Y_components
import matplotlib.tri as tri


run_id = 0

Setting up the Matmodlab notebook environment


In [2]:
def align_pi_plane_with_axes_rot():
    """
    Returns a matrix that rotates the pi plane's normal to be the z axis
    i.e., a slice of pi plane becomes the xy plane after rotation
    """
    pi_vector = np.array([1, 1, 1]) / np.sqrt(3.)
    # wanted_vector = np.array([1, 0, 0])
    wanted_vector = np.array([0, 0, 1])
    wanted_vector = wanted_vector / np.linalg.norm(wanted_vector)
    added = (pi_vector + wanted_vector).reshape([-1, 1])
    # from Rodrigues' rotation formula, more info here: https://math.stackexchange.com/a/2672702
    rot_mat = 2 * (added @ added.T) / (added.T @ added) - np.eye(3)
    return rot_mat


def align_axes_with_pi_plane_rot():
    """
    Returns a matrix that undoes the align_pi_plane_with_axes_rot rotation
    """
    return np.linalg.inv(align_pi_plane_with_axes_rot())

align_axes_with_pi_plane_rot = align_axes_with_pi_plane_rot()
align_pi_plane_with_axes_rot = align_pi_plane_with_axes_rot()

def run_ST_GPSR_TTM(property_dict, loads='ESS', components=(0.02, 0, 0), frames=50):
    mps1 = MaterialPointSimulator('ST_GPSR_TTM')
    mps1.material = ST_GPSR_TTM(**property_dict)
    mps1.run_step(loads,  components, frames=frames)
    return mps1

def draw_principal_axes(ax, length_of_axes=2.5, scale=100):
    """
    Draws principal axes on graph, change length of axes and scale
    """
    zero_pos = np.zeros(2)
    straight_up_pos = np.array([0, 1]) * length_of_axes
    bottom_right_pos = np.array([np.cos(-np.pi / 6.), np.sin(-np.pi / 6.)]) * length_of_axes
    bottom_left_pos = np.array([np.cos(7 * np.pi / 6.), np.sin(7 * np.pi / 6.)]) * length_of_axes

    ax.quiver(*zero_pos, *straight_up_pos, scale_units="xy", scale=1)
    ax.annotate(r"$\sigma_2$", straight_up_pos + np.array([0.25, -0.3]) * scale)

    ax.quiver(*zero_pos, *bottom_right_pos, scale_units="xy", scale=1)
    ax.annotate(r"$\sigma_1$", bottom_right_pos + np.array([-0.18, 0.45]) * scale)

    ax.quiver(*zero_pos, *bottom_left_pos, scale_units="xy", scale=1)
    # ax.annotate(r"$\sigma_3$", bottom_left_pos + np.array([0.1, -0.45]) * scale)
    ax.annotate(r"$\sigma_3$", bottom_left_pos + np.array([0.28, -0.4]) * scale)
    # ax.annotate(r"$\sigma_3$", bottom_left_pos + np.array([-0.18, -0.4]) * scale)

In [3]:
#""" Force deviatoric independence on a BINGO solution
def A_mapping(real_eqps  ):
    X_0 = real_eqps
    B_in = ((X_0) * (array([[ 0.4609263 ,  1.10732082, -1.64235896],
       [-5.78728213, -1.14002744, -7.41592524],
       [ 3.98790411,  2.2067136 ,  5.84326082]])) + array([[-0.03862302, -0.03187797,  0.20941924],
       [ 0.88568078,  0.49687034,  1.68504125],
       [-0.27601273, -0.15597089, -0.63898822]])) @ ((((X_0) * (array([[ 0.4609263 ,  1.10732082, -1.64235896],
       [-5.78728213, -1.14002744, -7.41592524],
       [ 3.98790411,  2.2067136 ,  5.84326082]])) + array([[-0.03862302, -0.03187797,  0.20941924],
       [ 0.88568078,  0.49687034,  1.68504125],
       [-0.27601273, -0.15597089, -0.63898822]])) @ (array([[ 0.70828347, -0.98725251,  0.28020231],
       [-1.25898614,  1.05867731,  0.20430282],
       [ 0.23715242,  0.0628872 , -0.30324894]]))) @ (((X_0) * (array([[ 0.4609263 ,  1.10732082, -1.64235896],
       [-5.78728213, -1.14002744, -7.41592524],
       [ 3.98790411,  2.2067136 ,  5.84326082]])) + array([[-0.03862302, -0.03187797,  0.20941924],
       [ 0.88568078,  0.49687034,  1.68504125],
       [-0.27601273, -0.15597089, -0.63898822]])) @ (array([[ 0.70828347, -0.98725251,  0.28020231],
       [-1.25898614,  1.05867731,  0.20430282],
       [ 0.23715242,  0.0628872 , -0.30324894]]))))
    #B_in *= (ISO_Y_0 + ISO_H*fict_eqps)
    B_in = align_axes_with_pi_plane_rot.T @ B_in @ align_axes_with_pi_plane_rot
    B_in[:, 2] = 0
    B_in[2, :] = 0
    B_in[2, 2] = 1
    B_in = align_axes_with_pi_plane_rot @ B_in @ align_axes_with_pi_plane_rot.T
    new_B = np.zeros((6,6))
    new_B[0:3, 0:3] = B_in
    new_B[3:, 3:] = np.eye(3) # Like the stuffness matrix, the shear comps are multiplied by ROOT2
    return new_B
#"""

# Shared params
E = 50e3
nu = .333

# Build the loading paths
rotation_angles = np.linspace(0,2*np.pi, 36)
axial_E = 0.1

pairs_master = [( 0, 1, 2 ), ( 1, 2, 0 ), ( 0, 2, 1 )]
pairs = [ pairs_master[ run_id ] ]
load_string_master = [ 'EES', 'SEE', 'ESE' ]
load_string = [ pairs_master[run_id] ]
pair_comps = [] 

load_comps = []
for i, pair in enumerate(pairs):
    local_comps = []
    local_load_comps = []
    a1 = pair[0]
    a2 = pair[1]
    s_index = pair[2]

    for ang in rotation_angles:
        local_load_comps.append(load_string[i])
        load_reference = [ 0, 0, 0 ]
        load_reference[a1] = axial_E * np.cos(ang)
        load_reference[a2] = axial_E * np.sin(ang)
        load_reference[s_index] = 0
        #print(F_reference)
        #rotation = rotations[rot](ang)
        # F_reference = F_reference.reshape(3,3)
        # F = np.dot(rotation, F_reference)
        # F = tuple(F.flatten())
        local_comps.append(load_reference)
    pair_comps.append(local_comps)
    load_comps.append(local_load_comps)

all_frames = 50

In [4]:
run = True
if run:
    results = []
    for comps_summary, load_summary in zip(pair_comps, load_comps):
        comp_results = []
        for comps, load in zip(comps_summary, load_summary):
            # Run ST_GPSR_TTM model
            pGPSR = {'E': E, 'Nu': nu,  "A_mapping": A_mapping}
            #print(comps, all_loads)
            try:
                mpsGPSR = run_ST_GPSR_TTM(pGPSR, frames=all_frames, components=comps, loads=load)
            except Exception as e:
                print(f'Failed on {comps_summary} and {load_summary}')
                print(e)
                continue
            comp_results.append(mpsGPSR)
        results.append(comp_results)


    with open("vpsc_results_12.pkl", "wb") as dill_file:
        dill.dump(results, dill_file)
else:
    with open("vpsc_results_12.pkl", "rb") as dill_file:
        results = dill.load(dill_file)

    # for mps in r:
    #     print(mps.df[['E.XX', 'E.YY', 'E.ZZ']].to_numpy()[-1,:])
    #     ax = mps.plot('E.XX', 'S.XX')
    #     mps.plot('E.XX', 'S.YY', ax=ax)
    #     mps.plot('E.XX', 'S.ZZ', ax=ax)
        


In [ ]:
eqps_vals = np.linspace(0, 0.1, 10)#np.array([0.000625*i for i in range(8) ])
if run_id == 0:
    plot_Y_components(results[0], pairs[0], eqps_vals, ['$\\sigma_{11}$', '$\\sigma_{22}$'])
elif run_id == 1:
    plot_Y_components(results[1], pairs[1], eqps_vals, ['$\\sigma_{22}$', '$\\sigma_{33}$'])
elif run_id == 2:
    plot_Y_components(results[2], pairs[2], eqps_vals, ['$\\sigma_{11}$', '$\\sigma_{33}$'])